In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle 
import time 
import datetime
import warnings
warnings.filterwarnings('ignore')
# import graft

In [2]:
with open('../fraudTrain.pkl', 'rb') as file:
    fraudTrain = pickle.load(file)    

In [3]:
def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result


In [4]:
df = fraudTrain[fraudTrain.is_fraud == 1]
fraud1_cc_num = df['cc_num'].unique()
df_fraud = fraudTrain[fraudTrain['cc_num'].isin(fraud1_cc_num)].reset_index()
df_group = df_fraud.groupby("cc_num")["is_fraud"].agg(['count', 'sum']).reset_index()
df_group.columns = ['cc_num', 'total_transactions', 'fraud_transactions']
first = df_fraud.groupby('cc_num')['first'].first().reset_index() 
last = df_fraud.groupby('cc_num')['last'].first().reset_index()
name = first['first'] + ' ' + last['last']
df_group['name'] = name
df_group.insert(1, 'name', df_group.pop('name'))
df_group

,cc_num,name,total_transactions,fraud_transactions
0,6.041621e+10,Mary Diaz,1242,9
1,6.042293e+10,Jeffrey Powers,1261,12
2,6.042310e+10,Jason Gray,423,10
3,6.042785e+10,Bradley Martinez,432,14
4,6.049060e+10,Karen Johnson,827,10
...,...,...,...,...
591,4.906630e+18,Charles Moreno,2103,11
592,4.911820e+18,Jeremy Chavez,9,9
593,4.956830e+18,Kenneth Robinson,2090,10
594,4.973530e+18,Mary Rodriguez,860,10


In [5]:
df_group.to_csv('df_group_ccnum', index=False)

In [6]:
df_fraud.is_fraud.mean()

0.009219716623428457

In [7]:
df_group['fraud_transactions'].sum()

6006

---

In [8]:
groups = df_fraud.groupby('cc_num')

In [ ]:
# edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
# edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
# edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
# np.save('edge_index_cc_num.npy', edge_index_list_plus_nparr)

# edge_index_list_plus_nparr

In [11]:
import graft

ModuleNotFoundError: No module named 'graph_tool'

In [9]:
import numpy as np
import torch_geometric
#import graft

def visualize_graph(df_fraud, c_sample=4.503100e+18, compute_time_difference=None, theta_multiplier=9, dr_opts=None):
    # cc_num에 해당하는 데이터 선택
    groups = df_fraud[df_fraud['cc_num'] == c_sample]

    # 시간 차이 계산
    edge_index = compute_time_difference(groups)
    edge_index = np.array(edge_index).astype(np.float64)
    edge_index = np.array(edge_index)[np.array(edge_index)[:, 0] != np.array(edge_index)[:, 1]]

    # Theta 계산
    theta = edge_index[:, 2].mean()

    # 가중치 설정
    edge_index[:, 2] = (np.exp(-edge_index[:, 2] / theta * theta_multiplier) != 1) * (np.exp(-edge_index[:, 2] / theta * theta_multiplier))

    edge_index = edge_index.tolist()

    # 인덱스 선택 및 사기 여부 추출
    selected_indices = df_fraud[df_fraud['cc_num'] == c_sample].index
    is_fraud_values = df_fraud.loc[selected_indices, 'is_fraud'].tolist()

    # 그래프 데이터 생성
    g = torch_geometric.data.Data(
        edge_index=np.array(edge_index)[:, :2],
        edge_attr=np.array(edge_index)[:, -1],
        x=df_fraud.loc[selected_indices, 'amt'].tolist(),
        y=is_fraud_values
    )

    # 노드의 고유값 추출
    unique_nodes = set(g.edge_index.reshape(-1).tolist())
    dict_map = {n: i for i, n in enumerate(unique_nodes)}

    # 인덱스 변환
    source, target = np.array(edge_index)[:, 0], np.array(edge_index)[:, 1]
    edge_index_selected = torch.tensor([[dict_map[s] for s in source.tolist()], [dict_map[t] for t in target.tolist()]])

    # 그래프 데이터 업데이트
    g = torch_geometric.data.Data(
        edge_index=edge_index_selected,
        edge_attr=np.array(edge_index)[:, -1],
        x=df_fraud.loc[selected_indices, 'amt'].tolist(),
        y=is_fraud_values
    )

    # 그래프 시각화
    if dr_opts is None:
        dr_opts = {'output_size': (150, 150), 'edge_marker_size': 1}

    graft.graph.plot_undirected_weighted(
        g,
        node_color=g.y,
        node_size=g.x,
        edge_weight_text=False,
        edge_weight_width_scale=3.0,
        draw_options=dr_opts,
    )

# 함수 호출 예시
#visualize_graph(df_fraud, c_sample=4503100000000000000, compute_time_difference=compute_time_difference, dr_opts=dr_opts)


In [ ]:
c_sample = 4.503100e+18

In [ ]:
groups = df_fraud[df_fraud['cc_num'] == c_sample]

edge_index = compute_time_difference(groups)
edge_index = (np.array(edge_index)).astype(np.float64)
edge_index = np.array(edge_index)[np.array(edge_index)[:,0] != np.array(edge_index)[:,1]]
theta = edge_index[:,2].mean()
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta*9)!= 1)*(np.exp(-edge_index[:,2]/theta*9))
edge_index = edge_index.tolist()
selected_indices1 = df_fraud[df_fraud['cc_num'] ==  c_sample].index
is_fraud_values1 = df_fraud.loc[selected_indices1, 'is_fraud'].tolist()
g = torch_geometric.data.Data(
    edge_index = np.array(edge_index)[:,:2],
    edge_attr = np.array(edge_index)[:,-1],  # weight
    x = df_fraud.loc[selected_indices1, 'amt'].tolist(),
    y = df_fraud.loc[selected_indices1, 'is_fraud'].tolist()
)



unique_nodes = set(g.edge_index.reshape(-1).tolist())
dict_map = {n:i for i,n in enumerate(unique_nodes)}
source, target = np.array(edge_index)[:,0], np.array(edge_index)[:,1]
edge_index_selected2 = torch.tensor([[dict_map[s] for s in source.tolist()],[dict_map[t] for t in target.tolist()]])

g = torch_geometric.data.Data(
    edge_index = edge_index_selected2,
    edge_attr = np.array(edge_index)[:,-1],
    x = df_fraud.loc[selected_indices1, 'amt'].tolist(),
    y = df_fraud.loc[selected_indices1, 'is_fraud'].tolist()
)

dr_opts = { 
    'output_size': (150,150),
    'edge_marker_size': 1,
  #'output':"ten-nodes.pdf",
}
graft.graph.plot_undirected_weighted(
    g,
    node_color=g.y,
    node_size=g.x,
    edge_weight_text=False,
    edge_weight_width_scale=3.0,
    draw_options= dr_opts,
)


